In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json

# 접속 정보 로딩 (옵션: db-config.json 파일이 있을 경우)
with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# air_quality 테이블 데이터 읽어오기
query = "SELECT * FROM air_quality"
air_quality = pd.read_sql(query, con=engine)
query2 = "SELECT * FROM air_pollution"
air_pollution = pd.read_sql(query2, con=engine)
query3 = "SELECT * FROM vehicle_reg_stats"
vehicle = pd.read_sql(query3, con=engine)

In [2]:
# ▶ 결과 확인
air_quality.head()

,id,datetime,region,pm10,pm25
0,1,2015-12-31 23:00:00,평균,93.0,62.0
1,2,2015-12-31 23:00:00,강남구,107.0,70.0
2,3,2015-12-31 23:00:00,강동구,91.0,51.0
3,4,2015-12-31 23:00:00,강북구,92.0,64.0
4,5,2015-12-31 23:00:00,강서구,96.0,51.0


In [3]:
air_quality.isnull().sum()

id              0
datetime        0
region          0
pm10        73631
pm25        70223
dtype: int64

In [4]:
#월별 & 지역별 평균으로 결측치 대체

air_quality['datetime'] = pd.to_datetime(air_quality['datetime'])

air_quality['year_month'] = air_quality['datetime'].dt.to_period('M')

cols_to_fill = ['pm10', 'pm25']

for col in cols_to_fill:
    air_quality[col] = air_quality.groupby(['year_month', 'region'])[col].transform(
        lambda x: x.fillna(x.mean())
    )

air_quality.isnull().sum()

id              0
datetime        0
region          0
pm10          720
pm25          720
year_month      0
dtype: int64

In [5]:
# 연월 평균으로 결측치 보완

for col in ['pm10', 'pm25']:
    air_quality[col] = air_quality.groupby('year_month')[col].transform(
        lambda x: x.fillna(x.mean())
    )

air_quality.isnull().sum()

id            0
datetime      0
region        0
pm10          0
pm25          0
year_month    0
dtype: int64

In [6]:
air_quality.drop(columns='id', inplace=True)
air_quality.drop(columns='year_month', inplace=True)

air_quality['pm10'] = air_quality['pm10'].astype(int)
air_quality['pm25'] = air_quality['pm25'].astype(int)

air_quality.head()

,datetime,region,pm10,pm25
0,2015-12-31 23:00:00,평균,93,62
1,2015-12-31 23:00:00,강남구,107,70
2,2015-12-31 23:00:00,강동구,91,51
3,2015-12-31 23:00:00,강북구,92,64
4,2015-12-31 23:00:00,강서구,96,51


In [7]:
# 월별 & 지역별 데이터 평균
# datetime 형식 변환
air_quality['datetime'] = pd.to_datetime(air_quality['datetime'])

#날짜를 대표일 포함한 형식으로 변환(2015-01-01)
air_quality['datetime'] = air_quality['datetime'].dt.to_period('M').dt.to_timestamp()

In [8]:
# 월별&지역별 평균

quality_monthly = (
    air_quality[air_quality['region'] != '평균'] #평균행제외
    .groupby(['region', 'datetime'])[['pm10', 'pm25']]
    .mean()
    .reset_index()
)

In [9]:
quality_monthly['pm10'] = quality_monthly['pm10'].astype(int)
quality_monthly['pm25'] = quality_monthly['pm25'].astype(int)

quality_monthly.head()

,region,datetime,pm10,pm25
0,강남구,2012-01-01,63,41
1,강남구,2012-02-01,50,27
2,강남구,2012-03-01,47,24
3,강남구,2012-04-01,52,21
4,강남구,2012-05-01,52,25


In [10]:
air_pollution.head()

,id,datetime,region,no2,co,so2,o3
0,14907,2015-01-01 01:00:00,평균,0.0099,0.36,0.0054,0.0210
1,14908,2015-01-01 02:00:00,평균,0.0096,0.37,0.0055,0.0201
2,14909,2015-01-01 03:00:00,평균,0.0090,0.38,0.0054,0.0197
3,14910,2015-01-01 04:00:00,평균,0.0074,0.35,0.0053,0.0203
4,14911,2015-01-01 05:00:00,평균,0.0074,0.34,0.0052,0.0195


In [11]:
air_pollution.isnull().sum()

id          0
datetime    0
region      0
no2         0
co          0
so2         0
o3          0
dtype: int64

In [12]:
# 월별 & 지역별 데이터 평균
# datetime 형식 변환
air_pollution['datetime'] = pd.to_datetime(air_pollution['datetime'])

#날짜를 대표일 포함한 형식으로 변환(2015-01-01)
air_pollution['datetime'] = air_pollution['datetime'].dt.to_period('M').dt.to_timestamp()

In [13]:
air_pollution.drop(['id'], axis=1, inplace=True)

In [14]:
# 월별&지역별 평균

pollution_monthly = (
    air_pollution[air_pollution['region'] != '평균'] #평균행제외
    .groupby(['region', 'datetime'])[['no2', 'co', 'so2', 'o3']]
    .mean()
    .reset_index()
)

In [15]:
air_pollution['no2'] = air_pollution['no2'].round(4)
air_pollution['co'] = air_pollution['co'].round(2)
air_pollution['so2'] = air_pollution['so2'].round(4)
air_pollution['o3'] = air_pollution['o3'].round(4)


In [16]:
pollution_monthly.head()

,region,datetime,no2,co,so2,o3
0,강남구,2015-01-01,0.040934,0.587291,0.006493,0.010877
1,강남구,2015-02-01,0.039368,0.531554,0.006356,0.016963
2,강남구,2015-03-01,0.044098,0.533225,0.006445,0.022103
3,강남구,2015-04-01,0.037098,0.422153,0.005117,0.025397
4,강남구,2015-05-01,0.034963,0.399352,0.005917,0.029238


In [17]:
vehicle.head()

,id,datetime,region,fuel_type,use_type,reg_count
0,136,2017-01-01,강원,경유,사업용,12362.0
1,137,2017-01-01,광주,전기,비사업용,240.0
2,138,2017-01-01,인천,전기,사업용,6.0
3,139,2017-01-01,인천,전기,비사업용,210.0
4,140,2017-01-01,대구,전기,사업용,53.0


In [18]:
seoul = vehicle[vehicle['region'] == '서울']
seoul.head()

,id,datetime,region,fuel_type,use_type,reg_count
8,144,2017-01-01,서울,전기,사업용,706.0
9,145,2017-01-01,서울,전기,비사업용,883.0
97,233,2017-01-01,서울,경유,사업용,70175.0
98,234,2017-01-01,서울,경유,비사업용,1042523.0
104,240,2017-01-01,서울,휘발유,사업용,18396.0


In [19]:
#사업용 비사업용 합계

summary = seoul.groupby(['datetime', 'region', 'fuel_type'], as_index=False)['reg_count'].sum()
summary.head()

,datetime,region,fuel_type,reg_count
0,2017-01-01,서울,경유,1112698.0
1,2017-01-01,서울,전기,1589.0
2,2017-01-01,서울,휘발유,1600398.0
3,2017-02-01,서울,경유,1115138.0
4,2017-02-01,서울,전기,1649.0


In [20]:
seoul_wide = summary.pivot(index='datetime', columns='fuel_type', values='reg_count')

seoul_wide = seoul_wide.reset_index().rename_axis(columns=None)

seoul_wide.head()

,datetime,경유,전기,휘발유
0,2017-01-01,1112698.0,1589.0,1600398.0
1,2017-02-01,1115138.0,1649.0,1600615.0
2,2017-03-01,1116960.0,1692.0,1601053.0
3,2017-04-01,1119341.0,1714.0,1602253.0
4,2017-05-01,1122534.0,1932.0,1602947.0


In [21]:
seoul_wide = seoul_wide.rename(columns={
    '경유': 'diesel',
    '전기': 'electric',
    '휘발유': 'gasoline'
})

seoul_wide.head()

,datetime,diesel,electric,gasoline
0,2017-01-01,1112698.0,1589.0,1600398.0
1,2017-02-01,1115138.0,1649.0,1600615.0
2,2017-03-01,1116960.0,1692.0,1601053.0
3,2017-04-01,1119341.0,1714.0,1602253.0
4,2017-05-01,1122534.0,1932.0,1602947.0


In [22]:
seoul_wide['diesel'] = seoul_wide['diesel'].astype(int)
seoul_wide['electric'] = seoul_wide['electric'].astype(int)
seoul_wide['gasoline'] = seoul_wide['gasoline'].astype(int)

seoul_wide.head()

,datetime,diesel,electric,gasoline
0,2017-01-01,1112698,1589,1600398
1,2017-02-01,1115138,1649,1600615
2,2017-03-01,1116960,1692,1601053
3,2017-04-01,1119341,1714,1602253
4,2017-05-01,1122534,1932,1602947


In [23]:
pollution_monthly['datetime'] = pd.to_datetime(pollution_monthly['datetime'])
quality_monthly['datetime'] = pd.to_datetime(quality_monthly['datetime'])
seoul_wide['datetime'] = pd.to_datetime(seoul_wide['datetime'])

In [24]:
print("✅ pollution_monthly:", pollution_monthly['datetime'].min().date(), "→", pollution_monthly['datetime'].max().date())
print("✅ quality_monthly:", quality_monthly['datetime'].min().date(), "→", quality_monthly['datetime'].max().date())
print("✅ seoul_wide:", seoul_wide['datetime'].min().date(), "→", seoul_wide['datetime'].max().date())

✅ pollution_monthly: 2015-01-01 → 2025-06-01
✅ quality_monthly: 2012-01-01 → 2024-12-01
✅ seoul_wide: 2017-01-01 → 2025-05-01


In [25]:
start_date = '2017-01-01'
end_date = '2024-12-01'

pollution_monthly = pollution_monthly[(pollution_monthly['datetime'] >= start_date) & (pollution_monthly['datetime'] <= end_date)]
quality_monthly = quality_monthly[(quality_monthly['datetime'] >= start_date) & (quality_monthly['datetime'] <= end_date)]
seoul_wide = seoul_wide[(seoul_wide['datetime'] >= start_date) & (seoul_wide['datetime'] <= end_date)]

In [26]:
monthly_merged = pd.merge(pollution_monthly, quality_monthly, on=['region', 'datetime'], how='inner')

final_merged = pd.merge(monthly_merged, seoul_wide, on='datetime', how='left')

In [27]:
cols_to_round = ['no2','so2', 'o3']
final_merged[cols_to_round] = final_merged[cols_to_round].round(3)
final_merged['co'] = final_merged['co'].round(1)

In [28]:
final_merged['datetime'] = pd.to_datetime(final_merged['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [29]:
final_merged

,region,datetime,no2,co,so2,o3,pm10,pm25,diesel,electric,gasoline
0,강남구,2017-01-01 00:00:00,0.034,0.6,0.005,0.012,57,31,1112698,1589,1600398
1,강남구,2017-02-01 00:00:00,0.036,0.6,0.005,0.016,51,29,1115138,1649,1600615
2,강남구,2017-03-01 00:00:00,0.039,0.5,0.005,0.021,63,44,1116960,1692,1601053
3,강남구,2017-04-01 00:00:00,0.032,0.4,0.005,0.029,57,24,1119341,1714,1602253
4,강남구,2017-05-01 00:00:00,0.027,0.3,0.005,0.032,62,23,1122534,1932,1602947
...,...,...,...,...,...,...,...,...,...,...,...
2395,중랑구,2024-08-01 00:00:00,0.010,0.3,0.002,0.038,24,17,969642,80474,1650657
2396,중랑구,2024-09-01 00:00:00,0.010,0.3,0.002,0.031,16,9,964816,81088,1650101
2397,중랑구,2024-10-01 00:00:00,0.014,0.3,0.002,0.026,19,11,958780,81599,1647591
2398,중랑구,2024-11-01 00:00:00,0.021,0.5,0.002,0.025,26,15,953875,82824,1645768


In [30]:
final_merged.to_csv('전기차용_데이터셋.csv', index=False, encoding='utf-8')